In [120]:
%%capture
%run 01_hash.ipynb
%run 02_transaction.ipynb

In [121]:
import subprocess as sub

## 99_contract

Our goal is to be able to run a small piece of code in a tx, just like Ethereum. This piece of code is also called a smart contract (sc). The sc is like another user, but in addition to the balance it also has storage for variables and the sc itself. 

The smart contract should be able to access the current tx and block information.

## TODO:

- Inject block info

The smart contract that we are going to run contains the following code:

In [319]:
code = """
def init():
    r = 0
    s = 2
    h = 'hello how are you?'

r += 5
time = _tx['time'] + 'XXXXX'
"""

Temporary file name before execution starts.

In [320]:
F = 'temp.py'

This is injected into every execution to get all the variables used in the contract.

In [321]:
PRINT_VARS = ['local = set(locals())',
              'local_vars = {}',
              'for n in local:',
              '    if not n.startswith(\'_\'):',
              '         if \'str\' in str(type(eval(n))): print(n, f\'\"{eval(n)}\"\', \'str\')',
              '         else: print(n, eval(n), \'int\')']

All vars used will be stored in the `STORAGE` dict.

In [322]:
STORAGE = {}

Inject transaction `tx` and `block` into the execution. These can be accessed in the sc.

In [323]:
def inject(tx, code, block=None): 
    tx = tx.json().replace('true', 'True')
    return '_tx = '+tx+'\n'+code

In [324]:
def save(code):
    with open(F, 'w+') as f: f.write(code)

In [325]:
def run():
    r = sub.check_output(f'python3 {F}', shell=True)
    return r.decode().split('\n')[:-1]

Stores all variables in `STORAGE`.

In [326]:
def store(r):
    for a in r:
        if 'str' in a: b = a.split('"')
        else         : b = a.split(' ')
        STORAGE[b[0].strip()] = [b[1], b[2].strip()]

The deployment will run all the code in `init()`. The rest will run in `execution`.

In [327]:
def deployable(code):
    init = []
    lines = code.split('\n')
    for i,l in enumerate(lines):
        if 'init' in l: 
            for j in range(i+1, len(lines)):
                if lines[j][:1] == ' ': init.append(lines[j].strip())
                else                  : break
    init.extend(PRINT_VARS)
    return '\n'.join(l for l in init)

In [328]:
def deploy(code, tx):
    dc = deployable(code)
    ic = inject(tx, dc)
    save(ic)
    r  = run()
    store(r)

deploy(code, tx1); STORAGE

{'r': ['0', 'int'], 's': ['2', 'int'], 'h': ['hello how are you?', 'str']}

Execution will run everything in the sc except what is inside `init()`.

In [329]:
def executable(code):
    c = []
    for k,v in STORAGE.items():
        if v[1] == 'str': c.append(f'{k} = "{v[0]}"')
        else            : c.append(f'{k} = {v[0]}')
    lines = code.split('\n')
    in_init = False
    for l in lines:
        if 'init' in l: in_init = True; continue
        if in_init:
            if l[:1] == ' ': continue
            else           : in_init = False
        else: c.append(l)
    c.extend(PRINT_VARS)
    return '\n'.join(l for l in c)

In [330]:
def exe(code, tx):
    ic  = inject(tx, code)
    exe = executable(ic)
    save(exe)
    r  = run()
    store(r)
    
exe(code, tx1); STORAGE

{'r': ['5', 'int'],
 's': ['2', 'int'],
 'h': ['hello how are you?', 'str'],
 'time': ['Mon Apr 12 02:23:37 2021XXXXX', 'str']}